In [1]:
import gymnasium as gym
import causal_gym
import numpy as np
from time import sleep

with open('windy_gridworld_optimal.npy', 'rb') as f:
    policy = np.load(f)

env = gym.make('causal_gym/WindyGridWorld-v0', size = 3)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
n1 = np.zeros([env.size, env.size, env.action_space.n])
n2 = np.zeros([env.size, env.size, env.action_space.n, env.size, env.size])
ps = np.zeros([env.size, env.size, env.action_space.n, env.size, env.size])


/opt/homebrew/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.size to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.size` for environment variables or `env.get_wrapper_attr('size')` that will search the reminding wrappers.
  logger.warn(


In [3]:
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 1
delta = 0.1


# For plotting metrics
total_epochs = 0

for i in range(1, 50001):
    state, _ = env.reset()
    
    epochs, penalties, reward, = 0, 0, 0
    done = False
    terminated = False
    while not done and not terminated:
        action, next_state, reward, done, terminated, info = env.see()

        n1[state['agent'][0], state['agent'][1], action] += 1
        n2[state['agent'][0], state['agent'][1], action, next_state['agent'][0], next_state['agent'][1]] += 1

        state = next_state
        epochs += 1    

    total_epochs += epochs


/opt/homebrew/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.see to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.see` for environment variables or `env.get_wrapper_attr('see')` that will search the reminding wrappers.
  logger.warn(


In [4]:
policy = np.zeros([env.size, env.size], dtype=np.int8)
values = np.zeros([env.size, env.size])

for x in range(env.size):
    for y in range(env.size):
        for a in range(env.action_space.n):
            ps[x, y, a] = n2[x, y, a] / np.max([1, n1[x, y, a]])


theta = 0.1
max_iterations = 10
for j in range(max_iterations):
    err = 0.0
    new_values = np.zeros([env.size, env.size])
    for x in range(env.size):
        for y in range(env.size):   
            qtable = np.zeros(env.action_space.n)                 
            if x != int(env.size / 2) or y != int(env.size - 1):
                for a in range(env.action_space.n):
                    # Calculate the value of Q(s,a)
                    new_value = -1
                    for new_x in range(env.size):
                        for new_y in range(env.size):
                            new_value += ps[x, y, a, new_x, new_y] * gamma * values[new_x, new_y]

                    qtable[a] = new_value
                
            max_q = np.max(qtable)
            policy[x, y] = np.argmax(qtable)

            err = np.max([err, np.abs(values[x, y] - max_q)])
            new_values[x, y] = max_q

    values = new_values
    # Terminate if the value function has converged
    if err < theta:
        break

print(np.transpose(policy))
print(np.transpose(values))

[[0 0 0]
 [0 0 0]
 [0 0 1]]
[[-1. -1. -1.]
 [-1. -1. -1.]
 [-1.  0. -1.]]


In [5]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 10


env = gym.make('causal_gym/WindyGridWorld-v0', render_mode = 'human', size = 3)

for _ in range(episodes):
    state, _ = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    terminated = False
    
    while not done and not terminated:
        action = policy[state['agent'][0], state['agent'][1]]
        state, reward, done, terminated, info = env.step(action)
        clear_output(wait=True)
        epochs += 1

    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")

env.close()

Results after 10 episodes:
Average timesteps per episode: 9.9
